In [6]:
from dotenv import load_dotenv
import pandas as pd
import os
import boto3

In [7]:
test_file_path = "test.csv"
df = pd.read_csv(test_file_path)
df['english_instruction'].head()

0    Type a Python function to remove the first and...
1    Write a function to arrange a given matrix in ...
2    Write a function that counts the most common w...
3    Write a Python function to find the volume of ...
4    Write a function to divide a string into small...
Name: english_instruction, dtype: object

In [8]:
from botocore.exceptions import ClientError
import json
result_data = []

# Create a Bedrock Runtime client in the AWS Region of your choice.
client = boto3.client("bedrock-runtime", region_name="us-west-2")

# Set the model ID, e.g., Llama 3 70b Instruct.
model_id = "us.meta.llama3-2-3b-instruct-v1:0"

for index, row in df.iterrows():
    message = row['english_instruction']

    prompt = f"You are a helpful assistant that generates code snippets based on user instructions." \
                "The method signature should be according to the given in the example part.\n" \
                "Use the following guidlines in generation: \n" \
                " - Provide the code only\n" \
                " - Do not output any other texts or test cases results.\n" \
                " - Include the necessary import statements on the top even if they are usually not needed like - re, typing, itertools, Split etc. \n" \
                " - Do not include any comments in the code\n" \
                " - Do not forget to consider the edge cases\n" \
                " - Avoid recursion if possible\n" \
                " - Lastly, verify the code has no compilations error and contains the necessary imports.\n"
    

    # Embed the prompt in Llama 3's instruction format.
    formatted_prompt = f"""
    <|begin_of_text|><|start_header_id|>user<|end_header_id|>
    {prompt + message}
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """

    # Format the request payload using the model's native structure.
    native_request = {
        "prompt": formatted_prompt,
        "max_gen_len": 1024,
        "temperature": 0.5,
}

    # Convert the native request to JSON.
    request = json.dumps(native_request)

    try:
        # Invoke the model with the request.
        response = client.invoke_model(modelId=model_id, body=request)

    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
        exit(1)

    # Decode the response body.
    model_response = json.loads(response["body"].read())

    # Extract and print the response text.
    output = model_response["generation"]

    result_data.append({
        "id": row['id'],
        "response": output
    })
    print("Completed:", row['id'])

result_df = pd.DataFrame(result_data)

Completed: 1
Completed: 2
Completed: 3
Completed: 4
Completed: 5
Completed: 6
Completed: 7
Completed: 8
Completed: 9
Completed: 10
Completed: 11
Completed: 12
Completed: 13
Completed: 14
Completed: 15
Completed: 16
Completed: 17
Completed: 18
Completed: 19
Completed: 20
Completed: 21
Completed: 22
Completed: 23
Completed: 24
Completed: 25
Completed: 26
Completed: 27
Completed: 28
Completed: 29
Completed: 30
Completed: 31
Completed: 32
Completed: 33
Completed: 34
Completed: 35
Completed: 36
Completed: 37
Completed: 38
Completed: 39
Completed: 40
Completed: 41
Completed: 42
Completed: 43
Completed: 44
Completed: 45
Completed: 46
Completed: 47
Completed: 48
Completed: 49
Completed: 50
Completed: 51
Completed: 52
Completed: 53
Completed: 54
Completed: 55
Completed: 56
Completed: 57
Completed: 58
Completed: 59
Completed: 60
Completed: 61
Completed: 62
Completed: 63
Completed: 64
Completed: 65
Completed: 66
Completed: 67
Completed: 68
Completed: 69
Completed: 70
Completed: 71
Completed: 72
C

In [9]:
# Create a json from the result_df
result_json = result_df.to_json(orient="records", indent=4)
# save json in the data_folder os.path.join(data_folder, "submission.json")
with open("llama_3.2_3B_submission.json", "w") as json_file:
    json_file.write(result_json)